#### Lambda en la cual ingresamos la ingesta de datos mediante eventos

In [ ]:
import json
import boto3
from botocore.exceptions import ClientError

# Crear el cliente DynamoDB
dynamodb = boto3.resource('dynamodb')

# Especifica la tabla de DynamoDB
table = dynamodb.Table('Peliculas_S3D2_xideral')

def lambda_handler(event, context):
    """
    Esta función Lambda ingresa los datos de una película a la tabla Peliculas_S3D2 de DynamoDB.
    """
    try:
        # Obtener los datos del evento (deberías enviarlos como un JSON en el evento)
        
        pelicula_id = event.get('pelicula_id')
        fecha_hora = event.get('fecha_hora')
        nombre = event.get('nombre')
        sala = event.get('sala')
        duracion = event.get('duracion')
        clasificacion = event.get('clasificacion')

        # Comprobamos que todos los campos necesarios estén presentes
        if not all([pelicula_id, fecha_hora, nombre, sala, duracion, clasificacion]):
            return {
                'statusCode': 400,
                'body': json.dumps('Faltan datos requeridos en el evento')
            }

        # Insertar la película en la tabla de DynamoDB
        response = table.put_item(
            Item={
                'pelicula_id': pelicula_id,
                'fecha_hora': fecha_hora,  # Lista de horarios
                'nombre': nombre,
                'sala': sala,
                'duracion': duracion,  # Duración en min
                'clasificacion': clasificacion
            }
        )

        # Retornar una respuesta exitosa
        return {
            'statusCode': 200,
            'body': json.dumps('Película ingresada con éxito'),
            'response': response
        }

    except ClientError as e:
        # Manejar errores de DynamoDB
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error al ingresar la película: {e.response["Error"]["Message"]}')
        }


#### Eventos

In [ ]:
# Evento 1
{
  "pelicula_id": "11",
  "fecha_hora": "2025-02-05 11:00:00",
  "nombre" : "Inception",
  "sala": "7",
  "duracion": "148",
  "clasificacion": "PG-13"
}

#Evento 2
{
  "pelicula_id": "12",
  "fecha_hora": "2025-02-06 12:00:00",
  "nombre" : "Interestelar",
  "sala": "5",
  "duracion": "169",
  "clasificacion": "PG-13"
}

#Evento 3
{
  "pelicula_id": "13",
  "fecha_hora": "2025-02-07 13:00:00",
  "nombre" : "Batman: El caballero de la noche",
  "sala": "1",
  "duracion": "152",
  "clasificacion": "PG-13"
}

#Evento 4
{
  "pelicula_id": "14",
  "fecha_hora": "2025-02-08 14:00:00",
  "nombre" : "Oppenheimer",
  "sala": "15",
  "duracion": "180",
  "clasificacion": "R"
}

#Evento 5
{
  "pelicula_id": "15",
  "fecha_hora": "2025-02-09 15:00:00",
  "nombre" : "Tenet",
  "sala": "13",
  "duracion": "150",
  "clasificacion": "PG-13"
}


# FERIA DE NOLAN

#### Test para comprobar que los eventos se guardan en la tabla de DynamoDB

In [ ]:
def obtener_todos_los_datos():
    try:
        # Realizar el escaneo de la tabla para obtener todos los ítems
        response = table.scan()
        
        # Verificar si hay ítems en la respuesta
        items = response.get('Items', [])
        
        if items:
            return items
        else:
            print("No se encontraron datos en la tabla.")
            return None
    except ClientError as e:
        print(f"Error al obtener los datos: {e.response['Error']['Message']}")
        return None

# Llamar a la función para obtener todos los datos
datos = obtener_todos_los_datos()

# Imprimir los datos obtenidos
if datos:
    print("Datos encontrados en la tabla:")
    for item in datos:
        print(item)